In [ ]:
import numpy as np
import rasterio
from sklearn.preprocessing import MinMaxScaler



In [ ]:
def read_raster(file_path):
    """Read a raster file and return the data and metadata."""
    with rasterio.open(file_path) as src:
        data = src.read(1)  # Read the first band
        metadata = src.meta
    return data, metadata

def normalize_raster(data):
    """Normalize raster data to the range [0, 1]."""
    scaler = MinMaxScaler()
    data_flat = data.flatten().reshape(-1, 1)
    normalized_flat = scaler.fit_transform(data_flat)
    return normalized_flat.reshape(data.shape)

def ahp_weights(criteria_weights):
    """Calculate normalized AHP weights from criteria weights."""
    total = sum(criteria_weights)
    return [weight / total for weight in criteria_weights]

def topsis(data, weights):
    """Perform TOPSIS analysis."""
    norm_data = data / np.sqrt((data ** 2).sum(axis=0))
    weighted_data = norm_data * weights

    ideal_best = np.max(weighted_data, axis=0)
    ideal_worst = np.min(weighted_data, axis=0)

    distance_best = np.sqrt(((weighted_data - ideal_best) ** 2).sum(axis=1))
    distance_worst = np.sqrt(((weighted_data - ideal_worst) ** 2).sum(axis=1))

    scores = distance_worst / (distance_best + distance_worst)
    return scores

def weighted_overlay(rasters, weights):
    """Perform weighted overlay of rasters."""
    combined = np.zeros_like(rasters[0])
    for i, raster in enumerate(rasters):
        combined += raster * weights[i]
    return combined

def average_weighted(rasters):
    """Perform average weighted overlay of rasters."""
    return np.mean(rasters, axis=0)

def save_raster(output_path, data, metadata):
    """Save data to a new raster file."""
    metadata.update({"dtype": "float32"})
    with rasterio.open(output_path, "w", **metadata) as dst:
        dst.write(data, 1)

# File paths to input TIFFs
file_paths = ["reclassified_1.tif", "reclassified_2.tif", "reclassified_3.tif", "reclassified_4.tif", "reclassified_5.tif"]

# Read and normalize raster data
rasters = []
metadata = None
for file_path in file_paths:
    data, metadata = read_raster(file_path)
    normalized_data = normalize_raster(data)
    rasters.append(normalized_data)

# Define criteria weights (example values, sum must equal 1)
criteria_weights = [0.2, 0.3, 0.1, 0.25, 0.15]

# Normalize AHP weights
weights = ahp_weights(criteria_weights)

# Stack rasters into a single array for MCDM/TOPSIS analysis
stacked_rasters = np.stack(rasters, axis=2)
rows, cols, num_criteria = stacked_rasters.shape
stacked_flat = stacked_rasters.reshape(rows * cols, num_criteria)

# Perform TOPSIS analysis
scores_topsis = topsis(stacked_flat, weights)
suitability_map_topsis = scores_topsis.reshape(rows, cols)

# Perform Weighted Overlay analysis
weighted_map = weighted_overlay(rasters, weights)

# Perform Average Weighted analysis
average_weighted_map = average_weighted(rasters)

# Save output rasters
output_path_topsis = "habitat_suitability_topsis.tif"
output_path_weighted = "habitat_suitability_weighted_overlay.tif"
output_path_avg_weighted = "habitat_suitability_avg_weighted.tif"

save_raster(output_path_topsis, suitability_map_topsis, metadata)
save_raster(output_path_weighted, weighted_map, metadata)
save_raster(output_path_avg_weighted, average_weighted_map, metadata)

print(f"TOPSIS-based habitat suitability map saved to {output_path_topsis}")
print(f"Weighted overlay habitat suitability map saved to {output_path_weighted}")
print(f"Average weighted habitat suitability map saved to {output_path_avg_weighted}")
